<a href="https://colab.research.google.com/github/Pretzel-Solution/directional-sound-simulator/blob/master/simulate_directional_sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install sofasonix
#press "Mount Drive"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from SOFASonix import SOFAFile
import scipy.io.wavfile as wav


In [ ]:
loadsofa = SOFAFile.load("/content/drive/My Drive/binaural localization/HRIR_FULL2DEG.sofa")
data = loadsofa.data_ir
direction = loadsofa.SourcePosition
direction = direction[:,0:2]
#loadsofa.view()

Setting dimension 'r' from parameter 'ReceiverPosition'
Setting dimension 'e' from parameter 'EmitterPosition'
Setting dimension 'm' from parameter 'Data.IR'
Setting dimension 'n' from parameter 'Data.IR'
Inserting foreign parameter: 'GLOBAL:Author'
Inserting foreign parameter: 'GLOBAL:ListenerDescription'
Inserting foreign parameter: 'GLOBAL:ReceiverDescription'
Inserting foreign parameter: 'GLOBAL:SourceDescription'
Inserting foreign parameter: 'GLOBAL:RoomDescription'


In [ ]:
def get_hrir(az_wish, el_wish):
  m_altered = np.abs(direction[:,0]- az_wish) + np.abs(direction[:,1]- el_wish)
  m_min = np.amin(m_altered, axis=0)
  i_row = np.argwhere(m_altered == m_min)[0][0]
  return data[i_row][0], data[i_row][1], i_row

In [ ]:
def plot_hrir(az_wish, el_wish):
  hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish)
  t = np.linspace(0, len(hrir_l)/loadsofa.Data_SamplingRate, len(hrir_l))
  fig, ax = plt.subplots(2,1, figsize=(8,6))
  ax[0].plot(t, hrir_l, '-k')
  ax[0].set(title = 'HRIR left', xlabel='time in s', ylabel='amplitude')
  ax[1].plot(t, hrir_r, '-k')
  ax[1].set(title = 'HRIR right', xlabel= 'time in s', ylabel= 'amplitude')
  fig.suptitle('Azimuth: '+str(direction[i_row][0].round(1))+'° Elevation: '+str(direction[i_row][1].round(1))+'°')
  #fig.tight_layout()  


In [ ]:
def get_stereo_noise(az_wish, el_wish):
  duration = 3 #seconds
  sampling_rate = int(loadsofa.Data_SamplingRate[0]) #Hz
  sample_n = duration*sampling_rate
  t = np.linspace(0, duration, sample_n)
  hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish)

  noise = np.random.uniform(-1,1,sample_n)
  left = np.convolve(noise, hrir_l, mode='valid') # 'valid': avoid boundary effects; The convolution product is only given for points where the signals overlap completely
  right = np.convolve(noise, hrir_r, mode='valid')
  audio = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  wav.write('/content/drive/My Drive/binaural localization/WAV/noise['+str(direction[i_row][0].round(1))+', '+str(direction[i_row][1].round(1))+'].wav', sampling_rate, scaled)

In [ ]:
def get_stereo_audio(az_wish, el_wish):
  hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish)
  load_sprache = wav.read('/content/drive/My Drive/binaural localization/test_nic.wav')
  sprache = load_sprache[1]
  sampling_rate = load_sprache[0] 
  if  loadsofa.Data_SamplingRate[0] != sampling_rate:
    print("Sampling rate of recording and HRIR do not match!")
  left = np.convolve(sprache, hrir_l, mode='valid') # 'valid': avoid boundary effects; The convolution product is only given for points where the signals overlap completely
  right = np.convolve(sprache, hrir_r, mode='valid')
  audio = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  wav.write('/content/drive/My Drive/binaural localization/WAV/audio['+str(direction[i_row][0].round(1))+', '+str(direction[i_row][1].round(1))+'].wav', sampling_rate, scaled)

In [ ]:
def get_roundabout():
  load_sprache = wav.read('/content/drive/My Drive/binaural localization/hallo2.wav')
  sprache = load_sprache[1]
  sampling_rate = load_sprache[0]

  hrir_l, hrir_r, i_row = get_hrir(0, el_wish=0)
  left = np.convolve(sprache, hrir_l, mode='valid')
  right = np.convolve(sprache, hrir_r, mode='valid')
  audio = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
  for az_wish in np.arange(0,365,30):
    hrir_l, hrir_r, i_row = get_hrir(az_wish, el_wish=0)
    left = np.convolve(sprache, hrir_l, mode='valid')
    right = np.convolve(sprache, hrir_r, mode='valid')
    audio_n = np.hstack((left.reshape(-1,1),right.reshape(-1,1)))
    audio = np.vstack((audio,audio_n))
  scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
  wav.write('/content/drive/My Drive/binaural localization/audio_roundabout_hallo.wav', sampling_rate, scaled)

In [ ]:
np.arange(30,365,30)

array([ 30,  60,  90, 120, 150, 180, 210, 240, 270, 300, 330, 360])

In [ ]:
get_roundabout()

In [ ]:
for el in np.arange(0,360,10):
  get_stereo_audio(el, 0)

In [ ]:
get_stereo_noise(45,0)

In [ ]:
plot_hrir(az_wish=45, el_wish=1)

In [ ]:

get_wav(5,0)
# get_wav(360-45,0)

In [ ]:
hrir_l, hrir_r, i_row = get_hrir(az_wish=45, el_wish=0)
direction[i_row][0]

44.0